In [1]:
import random
from __future__ import division
import svmprediction
reload(svmprediction)
import feat
reload(feat)
from feat import extract_features
from feat import encode_categorical_features
from sklearn import svm

(features, labels, yard_labels, prog_labels) = extract_features(2009, 2014)


177245


In [111]:
import dimreduction
reload(dimreduction)

(svm_features, encoder) = encode_categorical_features(features)
(reduced_features, pca) = dimreduction.pca(svm_features.toarray(), svm_features.shape[1])
print sum(pca.explained_variance_ratio_), pca.noise_variance_

#import pickle
#pickle.dump( reduced_features, open( "features_pca1.p", "wb" ) )

[  9.97857167e-01   2.07908811e-03   4.91223361e-05   2.51415598e-06
   1.48316376e-06   1.10489155e-06   9.61470841e-07   8.55971418e-07
   5.81317881e-07   4.48773162e-07   1.60637834e-07   1.43574684e-07
   1.36248546e-07   1.35468678e-07   1.34570270e-07   1.32123573e-07
   1.29927650e-07   1.28087487e-07   1.27395847e-07   1.25634028e-07
   1.25355994e-07   1.24612715e-07   1.24334238e-07   1.23908792e-07
   1.22860736e-07   1.22159789e-07   1.21941059e-07   1.20917963e-07
   1.20479155e-07   1.19898068e-07   1.19614510e-07   1.18612414e-07
   1.18174625e-07   1.17393971e-07   1.16657459e-07   1.15671683e-07
   1.15213104e-07   1.14285708e-07   1.14131050e-07   1.13203640e-07
   1.12969074e-07   1.03840214e-07   9.97021493e-08   9.87777345e-08
   9.80832253e-08   9.73360338e-08   9.69721900e-08   9.61928191e-08
   9.56149184e-08   9.51319512e-08   9.49567517e-08   9.44042579e-08
   9.40659012e-08   9.37606957e-08   9.24627017e-08   9.22317337e-08
   9.14771330e-08   9.12285853e-08

In [113]:
sum(pca.explained_variance_ratio_[:3])

0.99998537724421344

In [112]:
import matplotlib.pyplot as plt

plt.clf()
x=range(1,len(pca.explained_variance_ratio_)+1)
plt.plot(x,pca.explained_variance_ratio_,marker='o')
plt.yscale('log')
plt.ylim([0.00000000000000000000000000000000001,10])
plt.ylabel('Variance ratio')
plt.xlabel('Component')
plt.title('Component variances')


plt.show()

In [110]:

plt.clf()
#x=range(1,len(pca.explained_variance_ratio_)+1)
x1=[]
y1=[]
x2=[]
y2=[]
for i,t in enumerate(reduced_features):
    if labels[i] == 1:
        x1.append(t[0])
        y1.append(t[1])
    else:
        x2.append(t[0])
        y2.append(t[1])

plt.scatter(x2,y2,marker='.',color='b',alpha=0.66,label='failure')
plt.scatter(x1,y1,marker='.',color='r',alpha=0.33,label='success')

plt.legend(loc=4)


plt.ylabel('Component 2')
plt.xlabel('Component 1')
plt.xlim([-1200,1000])
plt.title('Projection of first two components')


plt.show()

In [69]:
import matplotlib.pyplot as plt

plt.clf()
x=range(1,len(pca.explained_variance_ratio_)+1)
y = []
for i,v in enumerate(pca.explained_variance_ratio_):
    t = sum(pca.explained_variance_ratio_[:i+1])
    print t
    y.append(t)


plt.plot(x,y,marker='o')
plt.yscale('log')
plt.ylim([0.000000001,1])
plt.ylabel('Variance ratio')
plt.xlabel('Component')
plt.title('PCA on training data')


plt.show()

0.997857166796
0.999936254908
0.999985377244
0.9999878914
0.999989374564
0.999990479455
0.999991440926
0.999992296898
0.999992878216
0.999993326989
0.999993487627
0.999993631201
0.99999376745
0.999993902919
0.999994037489
0.999994169612
0.99999429954
0.999994427628
0.999994555023
0.999994680657
0.999994806013
0.999994930626
0.99999505496
0.999995178869
0.99999530173
0.99999542389
0.999995545831
0.999995666749
0.999995787228
0.999995907126
0.99999602674
0.999996145353
0.999996263527
0.999996380921
0.999996497579
0.999996613251
0.999996728464
0.999996842749
0.99999695688
0.999997070084
0.999997183053
0.999997286893
0.999997386596
0.999997485373
0.999997583456
0.999997680792
0.999997777765
0.999997873958
0.999997969572
0.999998064704
0.999998159661
0.999998254065
0.999998348131
0.999998441892
0.999998534355
0.999998626586
0.999998718064
0.999998809292
0.999998899398
0.999998988872
0.999999078011
0.999999166836
0.999999254694
0.999999342317
0.999999428543
0.999999514297
0.999999594963
0.99

In [89]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm
import feat
reload(feat)
(team_features,team_labels) = feat.get_team_features('NE', features, labels,feature_name='team')

(svm_features, encoder) = encode_categorical_features(team_features)

clf = tree.DecisionTreeClassifier(max_depth=2,class_weight='balanced')
cm = evaluate.clf_evaluate(clf,svm_features,team_labels)

evaluate.plot_tree(clf,encoder.feature_names_)
print cm

6048
**********************************
Recall: 50.8390918065 %
Precision: 53.2299741602 %
Accuracy: 68.5681216931 %
**********************************
[[3117  905]
 [ 996 1030]]


In [90]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm
import feat
reload(feat)
(team_features,team_labels) = feat.get_team_features('NO', features, labels,feature_name='opponent')

(svm_features, encoder) = encode_categorical_features(team_features)

classifier = svm.SVC(class_weight='balanced')
cm = evaluate.clf_evaluate(classifier,svm_features,team_labels)
print cm

5435
**********************************
Recall: 38.2746051033 %
Precision: 37.4109263658 %
Accuracy: 61.9135234591 %
**********************************
[[2735 1054]
 [1016  630]]


In [ ]:
encoder.vocabulary_

In [1]:
#import pickle
#reduced_features = pickle.load( open( "features_pca1.p", "rb" ) )

In [24]:
import evaluate
from sklearn import tree
reload(evaluate)
clf = tree.DecisionTreeClassifier(max_depth=8,class_weight='balanced')
cm = evaluate.clf_evaluate(clf,svm_features, labels)

#evaluate.plot_confusion_matrix(cm)

**********************************
Accuracy: 67.5923157212 %
Precision: 46.5255450788 %
Recall: 65.8565500889 %
F1: 54.5284706665 %
**********************************


In [52]:
from sklearn import tree
import evaluate
reload(evaluate)
import matplotlib.pyplot as plt

mae_list = []
rmse_list = []
x = [1,2,4,6,8,10,12,15]
for i in [1,2,4,6,8,10,12,15]:
    print i,'...'
    clf = tree.DecisionTreeRegressor(max_depth=i)
    (_,_,mae,rmse) = evaluate.reg_evaluate(clf,svm_features, yard_labels)
    mae_list.append(mae)
    rmse_list.append(rmse)
    

clf = tree.DecisionTreeRegressor()
diffs = evaluate.reg_evaluate(clf,svm_features,yard_labels)
print ""
#plt.hist(diffs,bins = 100)
#plt.xlabel('Difference [yards]')
#plt.show()

#evaluate.plot_tree(clf,encoder.feature_names_)


MAE: 7.26303421817 / RMSE: 11.7680667036



In [ ]:
plt.clf()
plt.plot()

In [67]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import svm

classifier = svm.SVC(class_weight='balanced')

X_train, X_test, y_train, y_test = train_test_split(reduced_features, labels)
# Run classifier
y_pred = classifier.fit(X_train, y_train).predict(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print cm

[[22986  8237]
 [ 7690  5399]]


In [ ]:
from sklearn.externals import joblib
joblib.dump(classifier, 'svm_pca6.pkl') 
#clf = joblib.load('svm_pca6.pkl') 

In [16]:
import evaluate
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
reload(evaluate)

clf = tree.DecisionTreeClassifier(max_depth=1,class_weight='balanced')
cm = evaluate.clf_evaluate(clf,svm_features,labels)
#,class_weight='balanced'
evaluate.plot_tree(clf,encoder.feature_names_)

print cm


**********************************
Accuracy: 69.1963101921 %
Precision: 47.891350965 %
Recall: 49.9646251219 %
F1: 48.906024818 %
**********************************
[[96517 28431]
 [26167 26130]]


In [3]:
#Example

#Train classifier
classifier.fit(svm_features, labels)  

#Predict result for play
X = defaultdict(float)
X['team'] = "GB"
X['opponent'] = "NYG"
X['time'] = 425
X['half'] = 1 
X['position'] = 10
X['half'] = 2
X['togo'] = 10
X['shotgun'] = 0
X['pass'] = 1
X['passlen'] = 'short'
X['side'] = 'left'
X['qbrun'] = 0
X = encoder.transform(X)

y_pred = clf.predict(X)
print prediction

NameError: name 'classifier' is not defined

In [19]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0,15,75)
y1 = []
y2 = []
y3 = []

for i in x:
    
    if i < 10.0:
        y3.append(0.0)
        y1.append(i/10.0)
        y2.append((i/10.0)**2)
    else:
        y3.append(1.0)
        y1.append(1.0)
        y2.append(1.0)

plt.clf()
plt.plot(x,y1,label='1st down')
plt.plot(x,y2,label='2nd down')
plt.plot(x,y3,label='3rd/4th down')
plt.ylim([0,1.1])
plt.xlim([0,12])
plt.legend(loc=2)
plt.xticks([0,5,10], ['0','togo/2','togo'])
plt.ylabel('Progress score')
plt.xlabel('Distance')
plt.title('Progress label (Version 1)')

plt.show()

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0,15,75)
y1 = []
y2 = []
y3 = []


for i in x:
    
    if i < 10.0:
        y3.append(0.0)
        y1.append((i*2)/10.0)
        y2.append(i/10.0)
    else:
        y3.append(1 + float(i - 10.0) / 10.0)
        y1.append(1 + float(i - 10.0) / 10.0)
        y2.append(1 + float(i - 10.0) / 10.0)

plt.clf()
plt.plot(x,y1,label='1st down')
plt.plot(x,y2,label='2nd down')
plt.plot(x,y3,label='3rd/4th down')
plt.ylim([0,2.2])
plt.xlim([0,15])
plt.legend(loc=2)
plt.xticks([0,5,10], ['0','togo/2','togo'])
plt.ylabel('Progress score')
plt.xlabel('Distance')
plt.title('Progress label (Version 1)')

plt.show()